In [1]:
from calendar import c
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlr import MLR


In [2]:
dataset = pd.read_csv("data/Folds5x2_pp.csv")
mlr = MLR(dataset, "PE")
mlr.learnParams(0.1)

ValueError: Data must be 1-dimensional